In [72]:
import datetime
import pandas as pd
import pygal
from IPython.display import display
from IPython.core.display import HTML

class Record(pd.DataFrame):

    def __init__(self, nutrition):
        super().__init__(nutrition)
        self.__record = nutrition
        
    def add(self, date, protein, carbon, fat):
        calorie = protein*4 + carbon*4 + fat*9
        new_record = pd.Series({"Date": pd.to_datetime("2023/01/01").date(), "Calorie": calorie, "Protein": protein, "Carbon": carbon, "Fat": fat})
        if self.__record[self.__record["Date"]==pd.to_datetime(date)].empty:
            new_data = self.__record.append(new_record, ignore_index=True)
        else:
            print("There has already been a record of the same date, please modify the record istead of adding a new one.")
            return Record(self.__record)
        return Record(new_data)


    def remove(self, date):
        
        record_to_remove = self.__record[self.__record["Date"]==pd.to_datetime(date)]
        if record_to_remove.empty:
            print("The record does NOT exist!")
            return Record(self.__record)
        else:
            print("Are you sure you want to delete the following record?\n",record_to_remove)
            
        while True:
            check = input("""Enter [1] for "YES"/ [2] for "NO" ("q" to quit):""")
            if check == "1":
                break
            elif check == "2" or check == "q":
                return Record(self.__record)
            else:
                print("Invalid input, please re-enter.")
                continue
                
        new_data = self.__record[~(self.__record["Date"] == pd.to_datetime(date))]
        return Record(new_data)
    
        
    def modify(self, date, protein, carbon, fat):
        
        row_to_modify = self.__record[self.__record["Date"]==pd.to_datetime(date)]
        if row_to_modify.empty:
            print("The record does NOT exist! Please check and try again!")
            return Record(self.__record)
        else:
            print("The record you are going to change:\n", row_to_modify)
            
        while True:
            check = input("""Enter [1] for "YES"/ [2] for "NO" ("q" to quit):""")
            if check == "1":
                break
            elif check == "2" or check == "q":
                return Record(self.__record)
            else:
                print("Invalid input, please re-enter.")
                continue
        
        if protein != '':
            self.__record.loc[row_to_modify.index[0], ("Protein",)] = protein
        if carbon != '':
            self.__record.loc[row_to_modify.index[0], ("Carbon",)] = carbon
        if fat != '':
            self.__record.loc[row_to_modify.index[0], ("Fat",)] = fat
        self.__record.loc[row_to_modify.index[0], ("Calorie",)] = self.__record.loc[row_to_modify.index[0], ("Protein",)]*4 + self.__record.loc[row_to_modify.index[0], ("Carbon",)]*4 + self.__record.loc[row_to_modify.index[0], ("Fat",)]*9
        new_data = self.__record
        return Record(new_data)   

    
    def show(self, start_date, end_date, indicator=1):
        
        condition = (self.__record["Date"] >= pd.to_datetime(start_date)) & (self.__record["Date"] <= pd.to_datetime(end_date))
        row_to_display = self.__record[condition]
        if row_to_display.empty:
            return "There is NO record between the selected dates." 
        
        # indicator == 1 means the user wants to display the record of Calories
        if indicator == 1:
            graph = pygal.Bar()
            graph.title = "Calorie Consumption Between {} and {}".format(start_date, end_date)
            graph.add("Calorie", row_to_display["Calorie"])
            graph.x_labels = row_to_display["Date"]
            
        # indicator != 1 means the user wants to display the record of other nutritions
        else:
            # pie chart will be used to display the nutritions of a single day
            if len(row_to_display) == 1:
                graph = pygal.Pie()
                graph.title = "Nutrition Consumption On {} (in g)".format(row_to_display["Date"].values[0])
                graph.add("Protein", row_to_display["Protein"].values[0])
                graph.add("Carbon", row_to_display["Carbon"].values[0])
                graph.add("Fat", row_to_display["Fat"].values[0])
                
            # stackedline will be used to display the nutritions of multiple days
            else:
                graph = pygal.StackedLine(fill=True)
                graph.title = "Nutrition Consumption From {} to {} (in g)".format(start_date, end_date)
                for each in row_to_display.keys()[2:]:
                    graph.add(each, row_to_display[each].to_list())
                graph.x_labels = row_to_display["Date"].to_list()
        
        # display the graph
        graph_svg = graph.render(is_unicode=True)
        display(HTML(graph_svg))

In [73]:
import pandas as pd

nutrition = pd.DataFrame({
    "Date": ["2023/01/01", "2023/01/02", "2023/01/03", "2023/01/04", "2023/01/05"],
    "Calorie": [2838, 1736, 2521, 1864, 2239],
    "Protein": [273, 176, 352, 61, 346],
    "Carbon": [156, 198, 203, 127, 231],
    "Fat": [283, 73, 199, 76, 230]})
nutrition["Date"] = pd.to_datetime(nutrition["Date"])

In [74]:
nutrition

,Date,Calorie,Protein,Carbon,Fat
0,2023-01-01,2838,273,156,283
1,2023-01-02,1736,176,198,73
2,2023-01-03,2521,352,203,199
3,2023-01-04,1864,61,127,76
4,2023-01-05,2239,346,231,230


In [75]:
r = Record(nutrition)
r["Calorie"][0]

C:\Users\xuxin\AppData\Local\Temp\ipykernel_23712\3045552549.py:11: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.__record = nutrition


2838

In [76]:
r.show("2023-01-01", "2023-01-05", 2)

In [77]:
r = r.add("2023/01/06",212,176,167)

C:\Users\xuxin\AppData\Local\Temp\ipykernel_23712\3045552549.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = self.__record.append(new_record, ignore_index=True)
C:\Users\xuxin\AppData\Local\Temp\ipykernel_23712\3045552549.py:11: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.__record = nutrition


In [12]:
r.remove("2023/1/6")

Are you sure you want to delete the following record?
         Date  Calorie  Protein  Carbon  Fat
5 2023-01-06     3055      212     176  167


Enter [1] for "YES"/ [2] for "NO" ("q" to quit): 1


,Date,Calorie,Protein,Carbon,Fat
0,2023-01-01,2838,273,156,283
1,2023-01-02,1736,176,198,73
2,2023-01-03,2521,352,203,199
3,2023-01-04,1864,61,127,76
4,2023-01-05,2239,346,231,230


In [78]:
r.modify("2023-01-05",188,129,231)

C:\Users\xuxin\AppData\Local\Temp\ipykernel_23712\3045552549.py:49: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  row_to_modify = self.__record[self.__record["Date"]==pd.to_datetime(date)]


The record you are going to change:
                   Date  Calorie  Protein  Carbon  Fat
4  2023-01-05 00:00:00     2239      346     231  230


Enter [1] for "YES"/ [2] for "NO" ("q" to quit): 1


C:\Users\xuxin\AppData\Local\Temp\ipykernel_23712\3045552549.py:11: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.__record = nutrition


,Date,Calorie,Protein,Carbon,Fat
0,2023-01-01 00:00:00,2838,273,156,283
1,2023-01-02 00:00:00,1736,176,198,73
2,2023-01-03 00:00:00,2521,352,203,199
3,2023-01-04 00:00:00,1864,61,127,76
4,2023-01-05 00:00:00,3347,188,129,231
5,2023-01-01,3055,212,176,167


In [48]:
d = input("...")
r[r["Date"]==pd.to_datetime(d)]

... 2023/01/06


,Date,Calorie,Protein,Carbon,Fat
5,2023-01-06,3055,212,176,167


In [71]:
d = pd.to_datetime("2023/01/01").date()
r[r["Date"]==d]

,Date,Calorie,Protein,Carbon,Fat


In [88]:
"2021/10/10">"2021/10/10"

False